In [1]:
import networkx as nx
import numpy as np 
import random

from dataclasses import dataclass, replace, field
from typing import Dict, Iterator, List, Optional, Set, Tuple, Union

import geopandas as gpd
import shapely
from shapely.geometry import Point
from shapely.strtree import STRtree

from falcomchain.helper import save_pickle, DataHandler
handler = DataHandler()
graph = handler.load_graph()
graphhh = handler.load_graphhh()
chicago = handler.load_chicago()
chicagooo = handler.load_chicagooo()
candidates = handler.load_candidates()  # set of nodes
geo_centers = handler.load_geo_centers()
geo_candidates = handler.load_geo_candidates()

In [ ]:
from falcomchain.candidates import Cells, assign_artificial_candidates
cells = Cells(
            blocks_gdf = chicagooo,
            graph = graphhh,  
            grid_size = 0.02,
            ) 

In [15]:
import random

cell_id = random.choice(list(cells.cells.keys()))
cell = cells.cells[cell_id]

In [21]:
block = random.choice(list(cell.blocks.keys()))

In [3]:
cells.total_needed_candidates()

2673.0

In [ ]:
a = assign_artificial_candidates(cells)

In [4]:
assigned = sum([len(cells.cells[cell_id].artificial_candidates) for cell_id in cells.cells])
missing = sum([cells.cells[cell_id].missing for cell_id in cells.cells])
needs = sum([cells.cells[cell_id].needs for cell_id in cells.cells])
reals = sum([len(cells.cells[cell_id].real_candidates) for cell_id in cells.cells])
print("reals", reals)
print(f"total need", needs)
print(f"Total missing artificial candidates: {missing}")
print(f"Total assigned artificial candidates: {assigned}")

reals 346
total need 2811
Total missing artificial candidates: 2465
Total assigned artificial candidates: 2433


In [5]:
fig = cells.plot()

In [7]:
r = sum(graphhh.nodes[node]['real_candidate'] for node in graphhh.nodes)
print(r)

352


In [9]:
2811 - 352

2459

In [11]:
cell_sums = {}
for cell_id in cells:
    cell = cells.cells[cell_id]
    for block in cell.blocks.keys():
        cell_sum = sum(cell.blocks[block].is_real for block in cell.blocks.keys())
    cell_sums[cell_id] = cell_sum
print("blockwwise", sum(cell_sums.values()))

sum_real = sum([len(cells.cells[cell_id].real_candidates) for cell_id in cells.cells.keys()])
print("over cells"  , sum_real)

blockwwise 399
over cells 346


In [ ]:


sum_artificial = sum([len(cells.cells[cell_id].artificial_candidates) for cell_id in cells.cells])
print(f"Total real candidates: {sum_real}")
print(f"Total artificial candidates: {sum_artificial}") 

for cell_id in cells:
    print(f"-------Cell ID: {cell_id}")
    print("real", len(cells.cells[cell_id].real_candidates))
    print("artificial", len(cells.cells[cell_id].artificial_candidates))

In [8]:
cand = []
for cell in cells.cells.values():
    cand.extend(cell.real_candidates)
    cand.extend(cell.artificial_candidates)
                
print(len(cand))

2804


In [9]:
noncand = set(graph.nodes) - set(cand)
noncand_zero = [node for node in noncand if graph.nodes[node]["population"] == 0]
print("noncand", len(noncand))
print("noncand_zero", len(noncand_zero))

noncand 36777
noncand_zero 4782


In [ ]:
fig = cells.plot()
fig = cells.plot_add_candidates(fig, candidate_type="artificial")
fig1 = cells.plot_add_candidates(fig, candidate_type="real")
fig1.show()

In [ ]:
cells.plot(block_type="non-candidate", candidates=cand)

In [ ]:
cells.plot(block_type="non-candidate", color_pop=True, candidates=cand)

In [ ]:
cells.plot(block_type="non-candidate", color_pop=True, candidates=cand)

In [ ]:
specials = {30185, 32588} # no travel time for these destinations. It is probably because of the street network 

# Make a copy and add a color column
geo_data = geo_data.copy()
geo_data["fill_color"] = geo_data.index.map(
    lambda idx: "yellow" if idx in specials else "lightgray"
)

fig = px.choropleth_mapbox(
    geo_data,
    geojson=geojson_data,
    locations=geo_data.index,
    color="fill_color",  # use custom color column
    color_discrete_map="identity",  # tells Plotly to use literal color names
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": -87.6298},
    zoom=9,
    opacity=0.6,
    width=1200,
    height=900,
)

fig.update_layout(
    title="Chicago Census Blocks (highlighted specials in yellow)",
    margin={"r":0,"t":40,"l":0,"b":0}
)

fig.show()


In [ ]:
fig = cells.plot()
fig = cells.plot_add_candidates(fig, candidate_type="artificial")
fig1 = cells.plot_add_candidates(fig, candidate_type="real")
fig1.show()

In [12]:
cells.assign_artificials_to_graph()

In [13]:
cells.save_OD_files(chicagooo)

Artificial candidates are assigned to the graph as node attributes.
Saved origins.csv and destinations.csv


/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/falcomchain/candidates/candidates.py:443: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [ ]:
from falcomchain.travel_time.r_wrapper import run_ttm

run_ttm(
    r_script_path="/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/falcomchain/travel_time/ttm.R",
    data_path="/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/data/network-data",
    origins_path="/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/data/network-data/origins.csv",
    dest_path="/Users/kirtisoglu/Documents/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/data/network-data/destinations.csv",
)
